<img src='https://repository-images.githubusercontent.com/121802384/c355bb80-7d42-11e9-9e0e-4729609f9fbc' alt='WRF-Hydro Logo' width="15%"/>

# Lesson 2 - Running WRF-Hydro

## Overview
In this lesson, we cover the basics of constructing and running a WRF-Hydro simulation using a prepared domain for the *gridded* routing configuration. 

For a detailed description of model see the [Technical Description](https://wrf-hydro.readthedocs.io/en/latest/index.html).

## Constructing a simulation with a prepared domain
In this section we describe the primary files needed to run a WRF-Hydro simulation. 

A WRF-Hydro simulation consists of the following major components:
-  executable/binary file
-  parameter files
-  domain files
-  forcing files
-  namelists

In this lesson we only cover basic descriptions of these elements. For an in-depth description of model see the [Technical Description](https://wrf-hydro.readthedocs.io/en/latest/index.html).

The WRF-Hydro model code, compiled executable/binary, and associated parameter table files were described in [Lesson 1](Lesson-1-compile.ipynb). 

**If you have not already completed Lesson 1, please stop and do so now**

Model run-time options are specified in two namelist files `hydro.namelist` and `namelist.hrldas`. These namelist files contain file path specifications, simulation duration, physics options, and output file selections, among others. We will cover many of these options in more depth in Lessons 4 and 5.   

## Orientation to the *example_case*
This lesson will use a prepared domain located in the `~/wrf-hydro-training/example_case` directory. The structure of the `example_case` directory serves as a good example of how to organize your domain files. If using another domain with this lesson, such as one you may have created in the geospatial pre-processing tutorial, it is **imperative** that the file names and directory structure match those described below.

If using an official WRF-Hydro training example case, there will be a study area map and a readme file that describes the geographic setting, the directory, and the files.

First lets take a look at the *study_map.PNG*.

![study_map.PNG](images/study_map.PNG)

Now, lets view the README file included with the domain for a brief description of the example case and its contents.

In [1]:
cd ~/wrf-hydro-training/example_case
cat README.txt

#Overview This test case includes prepared geospatial data and input files for a


sample domain (region of interest) and prepared forcing data.  This domain is a


small region (15km x 16km) encompassing the West Branch of the Croton River, NY,


USA (USGS stream gage 0137462010) during Hurricane Irene, 2011-08-26 to 2011-09-02.


The simulation begins with a restart from a spinup period from 2010-10-01 to


...

	-USGS-Obs.csv: csv files containing USGS 15 minute streamflow data for


	gages in the domain.


	-study_map.PNG: Study area map


	-namelist_patches.json: json file used by wrfhydropy python package for


	namelist parsing


Now lets take a look at the `example_case` directory.

In [2]:
ls

FORCING           Reach                         study_map.PNG


Gridded           ReachLakes                    supplemental


Gridded_no_lakes  USGS_obs.csv                  testing_diff_feature_ids.json


NWM               hrldas_namelist_patches.json


README.txt        hydro_namelist_patches.json


In this example case we have created several configurations for WRF-Hydro: *National Water Model (NWM)*, *Gridded*, and *Reach*. For detailed descriptions of routing and physics configurations see the [Technical Description](https://wrf-hydro.readthedocs.io/en/latest/model-code-config.html#wrf-hydro-run-time-options). These model configurations have different geospatial data associated with them and thus are represented by separate directories in the `~/wrf-hydro-training/example_case` directory. 

### FORCING
This directory contains all of the forcing data for our simulation. Note that there is only one `FORCING` directory. The same forcing data can be used with all the configurations.

### The Gridded configuration directory
For this lesson, we will be running the *Gridded* configuration. Now we will explore the `~/wrf-hydro-training/example_case/Gridded` directory.

In [3]:
ls Gridded

DOMAIN   croton_frxst_pts_csv.csv  namelist.hrldas


RESTART  hydro.namelist            referenceSim


The contents of this directory are described briefly in the `README.txt` file that we viewed earlier, but we will discuss them again here. For a more detailed description of these files beyond what will be described below, see the [Technical Description](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide)

**DOMAIN**: Directory containing all geospatial data and input files for
the Gridded routing option with lakes included

In [4]:
ls Gridded/DOMAIN

Fulldom_hires.nc                     LAKEPARM.nc    soil_properties.nc


GEOGRID_LDASOUT_Spatial_Metadata.nc  geo_em.d01.nc  wrfinput_d01.nc


GWBASINS.nc                          hydro2dtbl.nc


GWBUCKPARM.nc                        lake_shapes


| Filename | Description | Source | Required |
| ------------- | ------------- | ------------- | ------------- |
| Fulldom_hires.nc | High resolution full domain file. Includes all fields specified on the routing grid. | WRF-Hydro GIS pre-processing toolkit | Yes |
| GEOGRID_LDASOUT_Spatial_Metadata.nc | projection and coordinate information for the land surface model grid. | pre-processor | No, but allows for CF compliant outputs |
| GWBASINS.nc | 2D file defining the locations of groundwater basins on a grid | WRF-Hydro GIS pre-processing toolkit | When the baseflow bucket model is turned on and user defined mapping is off |
| GWBUCKPARM.nc | Groundwater parameter table containing bucket model parameters for each basin | WRF-Hydro GIS pre-processing toolkit | When the baseflow bucket model is turned on |
| LAKEPARM.nc | Lake parameter table containing lake model parameters for each catchment | WRF-Hydro GIS pre-processing toolkit | When lake and reservoir routing is turned on |
| geo_em.d01.nc | The data required to define the domain and geospatial attributes of a spatially-distributed, or gridded, 1-dimensional (vertical) land surface model (LSM) | GEOGRID utility in the WRF preprocessing system (WPS) | Yes |
| hydro2dtbl.nc | Spatially distributed parameter table for lateral flow routing within WRF-Hydro. | create_SoilProperties.R script (will also be automatically generated by WRF-Hydro) | When using spatially distributed terrain routing parameters |
| lake_shapes/ | supplemental shape files that define lakes | pre-processor | No |
| wrfinput_d01.nc | file including all necessary fields for the Noah-MP land surface model, but with spatially uniform initial conditions. Users should be aware that the model will likely require additional spin-up time when initialized from this file. | create_Wrfinput.R script | Yes |


**RESTART**: Directory containing model restart files

In [5]:
ls Gridded/RESTART

HYDRO_RST.2011-08-26_00:00_DOMAIN1  RESTART.2011082600_DOMAIN1


Restart files are an essential part of the WRF-Hydro modeling system. They are output on a fixed timestep specified by the user in the `namelist.hrldas` and `hydro.namelist` files, and represent a complete 'snapshot' of the model state at that time. These files can be used to restart a WRF-Hydro simulation from where the previous simulation terminated with all the model states intact.

When running a WRF-Hydro simulation, you may start your simulation with default initial conditions, referred to as a 'cold start'. When starting from a cold start, a model spinup period is needed to move the model state away from the default initial conditions to a more realistic, physically-based model state. Model output from the spinup period is generally not used for interpretation.

Restart files output at the end the spinup period can be used as the initial conditions for subsequent simulations, referred to as a 'warm start'. Simulations that start from a 'warm start' are generally the primary target for interpretation. 'Warm' and 'cold' starting the model will be discussed more in Lesson 4. 

### Namelists
Namelists are another key component of the WRF-Hydro modeling system and are the primary means of specifying inputs, outputs, and run-time options. There are two namelist files used by WRF-Hydro: `hydro.namelist` for the routing and hydrologic model and `namelist.hrldas` for the land surface model. 

**NOTE: These filenames are hard-coded into the model and can not be changed.**

For all official WRF-Hydro domains, namelists will be included with each of the model configurations. This is done so that a user can easily run each configuration with minimal setup, and they serve as a starting point for users to specify their own namelists for a given configuration.

There are different namelists for each configuration because certain namelist options are specific to the domain configuration used. We will cover the various namelist options in detail in Lesson 4, but for now we will simply view the namelist and use the default namelist provided with the Gridded configuration to construct our simulation.  

Take a moment and read through the two namelists below and note how all filepaths are relative to the current directory containing the namelist. This is the recommended way of specifying filepaths.

In [6]:
cat Gridded/hydro.namelist

&HYDRO_nlist


!!!! ---------------------- SYSTEM COUPLING ----------------------- !!!!


! Specify what is being coupled:  1=HRLDAS (offline Noah-LSM), 2=WRF, 3=NASA/LIS, 4=CLM


sys_cpl = 1


!!!! ------------------- MODEL INPUT DATA FILES ------------------- !!!!


...

! If on, specify the user-defined mapping file (e.g.: "spatialweights.nc")


!udmap_file = ""


/


&NUDGING_nlist


/


In [7]:
cat Gridded/namelist.hrldas

&NOAHLSM_OFFLINE


HRLDAS_SETUP_FILE = "./DOMAIN/wrfinput_d01.nc"


INDIR = "./FORCING"


SPATIAL_FILENAME = "./DOMAIN/soil_properties.nc"


OUTDIR = "./"


...

/


&CROCUS_nlist


  crocus_opt = 0   ! 0 model is off, 1 model is on


  act_lev = 40     ! 20-40 normal range, 1-50 acceptable


/


## Creating a simulation directory 
Now that we have covered the major functional elements that constitute a simulation, we will combine these elements and construct a simulation. This is done by placing the `FORCING`, `Gridded/DOMAIN`, `trunk/NDHMS/Run` directories and `namelist.hrldas` and `hydro.namelist` files together in a directory that will be our simulation directory. However, to save disk space it is often preferable to create symbolic links rather than copying the actual files. 

**NOTE: We will only use symbolic links with files that we will NOT be editing**    

![fig2.png](images/fig2.png)

In the following steps, we will construct our simulation directory.

**Step 1. Create simulation directory**

We will create a directory for our simulation

In [8]:
mkdir -p ~/wrf-hydro-training/output/lesson2/run_gridded_default
cd ~/wrf-hydro-training/output/lesson2/run_gridded_default

**Step 2. Copy model run files**

We will copy the required model run files from the *~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run* directory. These files are small so we will make actual copies rather than symbolic links in this case. Additionally, copies are preferred in this case because a user may want to edit the \*.TBL files and as stated previously symbolic links should not be used with files that we may edit.

In [9]:
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/build/Run/*.TBL .
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/build/Run/wrf_hydro .

ls 

CHANPARM.TBL  GENPARM.TBL  HYDRO.TBL  MPTABLE.TBL  SOILPARM.TBL  wrf_hydro


**Step 3. Symlink *DOMAIN* files**

We will create symbolic links to the required *domain* files from the `~/wrf-hydro-training/DOMAIN/Gridded` directory. These files can be large so we will make symbolic links rather than copying the actual files. 

In [10]:
ln -sf ~/wrf-hydro-training/example_case/FORCING .
ln -sf ~/wrf-hydro-training/example_case/Gridded/DOMAIN .
ln -sf ~/wrf-hydro-training/example_case/Gridded/RESTART .

ls 

CHANPARM.TBL  FORCING      HYDRO.TBL    RESTART       wrf_hydro


DOMAIN        GENPARM.TBL  MPTABLE.TBL  SOILPARM.TBL


**Step 4. Copy *namelist* files**

Because we are using the default prepared namelists from the example WRF-Hydro domain, we will copy those in as well. If you were using your own namelists, they would likely be edited and copied from elsewhere. These are small text files so we will make actual copies rather than symbolic links.

In [11]:
cp ~/wrf-hydro-training/example_case/Gridded/namelist.hrldas .
cp ~/wrf-hydro-training/example_case/Gridded/hydro.namelist .

ls 

CHANPARM.TBL  GENPARM.TBL  RESTART         namelist.hrldas


DOMAIN        HYDRO.TBL    SOILPARM.TBL    wrf_hydro


FORCING       MPTABLE.TBL  hydro.namelist


We have now constructed our simulation directory with all the requisite files. 

In the next section we will run the simulation. 

## Running WRF-Hydro using default run-time options
Now that we have constructed our simulation directory, we can run our simulation. For this we will be using the `mpirun` command, which has a number of arguments. For this simple case, we only need to supply one argument, the number of cores. This is done with the `-np` argument, and we will set it to 2 cores. 

We will pipe the output to a log file because running a simulation can generate a lot of standard output in the terminal.

In [12]:
mpirun -np 2 ./wrf_hydro &> run.log

If your simulation ran successfully, there should now be a large number of output files in the `~/wrf-hydro-training/output/lesson2/run_gridded_default`. We will describe these output files in more depth in Lesson 4. Additionally, detailed descriptions of the output files can be found in the [Technical Description](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide). 

List the contents of the run directory.

In [13]:
ls

201108260000.LDASOUT_DOMAIN1  201108291500.CHANOBS_DOMAIN1


201108260100.CHANOBS_DOMAIN1  201108291500.CHRTOUT_DOMAIN1


201108260100.CHRTOUT_DOMAIN1  201108291500.CHRTOUT_GRID1


201108260100.CHRTOUT_GRID1    201108291500.GWOUT_DOMAIN1


201108260100.GWOUT_DOMAIN1    201108291500.LAKEOUT_DOMAIN1


...

201108291400.CHRTOUT_GRID1    hydro.namelist


201108291400.GWOUT_DOMAIN1    namelist.hrldas


201108291400.LAKEOUT_DOMAIN1  run.log


201108291400.LDASOUT_DOMAIN1  wrf_hydro


201108291400.LSMOUT_DOMAIN1


There are also four important files for determining the success or failure of the run, diag_hydro.0000\*. The number of diag files is equal to the number of cores used for the run. These diag_hydro.0000\* files contain logs and diagnostics on the simulation run. 

You can check that your simulation ran successfully by examining the last line of the diag files, which should read  `The model finished successfully........`

In [14]:
tail -1 diag_hydro.00000

 The model finished successfully.......


# Next up - Basics of working with WRF-Hydro outputs
This concludes Lesson 2. In the next lesson, we will **briefly** discuss working with some of the output files. The output files from WRF-Hydro are standard netCDF4 files, and thus there are many way to work with these data. We simply cover a few Python libraries and commands that are needed for later lessons in this tutorial. 

[Lesson 3](Lesson-3-visualize.ipynb) is by no means a comprehensive guide to working with netCDF files. 

**IT IS BEST TO EITHER SHUTDOWN THIS LESSON OR CLOSE IT BEFORE PROCEEDING TO THE NEXT LESSON TO AVOID POSSIBLY EXCEEDING ALLOCATED MEMORY. Shutdown the lesson be either closing the browser tab for the lesson or selecting `Kernel -> Shut Down Kernel` in JupyterLab.**

© UCAR 2025